In [2]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine 
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import json

In [3]:
con = sqlite3.connect("BrewersAssociation/beer_project.sqlite")
BrewAssoc_df = pd.read_sql_query("SELECT * FROM breweries", con)
con.close()
BrewAssoc_df['Address_merg'] = BrewAssoc_df['address'] + "," + BrewAssoc_df['city'] + "," + BrewAssoc_df['state']
BrewAssoc_df.head()


,index,name,website,address,city,state,Address_merg
0,0,Patton Block Grill and Brewpub,,88-90 Public Square,Monmouth,IL,"88-90 Public Square,Monmouth,IL"
1,1,(405) Brewing Co,www.405brewing.com,1716 Topeka St,Norman,OK,"1716 Topeka St,Norman,OK"
2,2,(512) Brewing Co,www.512brewing.com,407 Radam Ln Ste F200,Austin,TX,"407 Radam Ln Ste F200,Austin,TX"
3,3,10 Torr Distilling and Brewing,www.10torr.com,490 Mill St,Reno,NV,"490 Mill St,Reno,NV"
4,4,10-56 Brewing Company,,2120 US-35,Knox,IN,"2120 US-35,Knox,IN"


In [ ]:
BrewAssoc_df['Address_merg'] = BrewAssoc_df['address'] + ", " + BrewAssoc_df['city'] + ", " + BrewAssoc_df['state']

geolocator = Nominatim(user_agent="BreweryLocs")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
BrewAssoc_df['location'] = BrewAssoc_df['Address_merg'].apply(geocode)


# BrewAssoc_df['point'] = BrewAssoc_df['Address_merg'].apply(lambda loc: tuple(loc.point) if loc else None)
BrewAssoc_df.head()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('192 E Main St, Canton, GA',), **{}).
Traceback (most recent call last):
  File "C:\Users\chris\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\chris\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\chris\anaconda3\lib\http\client.py", line 1344, in getresponse
    response.begin()
  File "C:\Users\chris\anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\chris\anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\chris\anaconda3\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\chris\anaconda3\lib\site-packages\urllib3\contrib\

RateLimiter swallowed an error after 2 retries. Called with (*('192 E Main St, Canton, GA',), **{}).
Traceback (most recent call last):
  File "C:\Users\chris\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\chris\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\chris\anaconda3\lib\http\client.py", line 1344, in getresponse
    response.begin()
  File "C:\Users\chris\anaconda3\lib\http\client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "C:\Users\chris\anaconda3\lib\http\client.py", line 267, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\chris\anaconda3\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\chris\anaconda3\lib\site-packages\urllib3\contrib\pyo

In [6]:

# BrewAssoc_df['point'] = BrewAssoc_df['location'].apply(lambda x: (x.latitude, x.longitude))
BrewAssoc_df['point'] = BrewAssoc_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

BrewAssoc_df.drop(columns=['location'])


KeyError: 'location'

In [109]:
BrewAssoc_df_split = BrewAssoc_df.dropna()
# BrewAssoc_df_point = BrewAssoc_df_split['point']
BrewAssoc_df_split

,index,name,website,address,city,state,Address_merg,location,point
0,0,Patton Block Grill and Brewpub,,88-90 Public Square,Monmouth,IL,"88-90 Public Square, Monmouth, IL","(88, Public Square, Monmouth, Warren County, I...","(40.91228195833334, -90.64796354166667, 0.0)"
1,1,(405) Brewing Co,www.405brewing.com,1716 Topeka St,Norman,OK,"1716 Topeka St, Norman, OK","(1716, Topeka Street, Norman, Cleveland County...","(35.25738890696931, -97.4681822227123, 0.0)"
3,3,10 Torr Distilling and Brewing,www.10torr.com,490 Mill St,Reno,NV,"490 Mill St, Reno, NV","(490, Mill Street, Midtown, Reno, Washoe Count...","(39.52570391836734, -119.80577028571429, 0.0)"
4,4,10-56 Brewing Company,,2120 US-35,Knox,IN,"2120 US-35, Knox, IN","(South Heaton Street, Knox, Starke County, Ind...","(41.2784872, -86.6206672, 0.0)"
6,6,1035 Brewing,,1035 S Central Ave,Sidney,MT,"1035 S Central Ave, Sidney, MT","(1035 Brewing, 1035, South Central Avenue, Fou...","(47.7070485, -104.1643577, 0.0)"
...,...,...,...,...,...,...,...,...,...
8399,8399,ZwanzigZ Brewing,www.zwanzigz.com,1038 Lafayette Ave,Columbus,IN,"1038 Lafayette Ave, Columbus, IN","(ZwanzigZ Pizza & Brewing, 1038, Lafayette Ave...","(39.20926005, -85.91909804812639, 0.0)"
8400,8400,Zwei Brewing Co,www.zweibruderbrewing.com,4612 S Mason St Ste 120,Fort Collins,CO,"4612 S Mason St Ste 120, Fort Collins, CO","(proVelo, 4612, South Mason Street, Arbor Plaz...","(40.5227407, -105.07859744351421, 0.0)"
8401,8401,Zydeco Brew Werks,,1902 E 7th Ave,Tampa,FL,"1902 E 7th Ave, Tampa, FL","(1902, East 7th Avenue, Ybor City, Tampa, Hill...","(27.960343795918366, -82.4378322857143, 0.0)"
8403,8403,Zymurgy Brewing Co,www.zymurgybrew.com,624 Main St East,Menomonie,WI,"624 Main St East, Menomonie, WI","(624, Main Street East, South Menomonie, Menom...","(44.876363, -91.92250454511311, 0.0)"


In [110]:
BrewAssoc_df_split[["latitude","longitue","Elevation"]] = pd.DataFrame(BrewAssoc_df_split["point"].tolist(), index=BrewAssoc_df_split.index)


C:\Users\chris\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,index,name,website,address,city,state,Address_merg,location,point,latitude,longitue,Elevation
0,0,Patton Block Grill and Brewpub,,88-90 Public Square,Monmouth,IL,"88-90 Public Square, Monmouth, IL","(88, Public Square, Monmouth, Warren County, I...","(40.91228195833334, -90.64796354166667, 0.0)",40.912282,-90.647964,0.0
1,1,(405) Brewing Co,www.405brewing.com,1716 Topeka St,Norman,OK,"1716 Topeka St, Norman, OK","(1716, Topeka Street, Norman, Cleveland County...","(35.25738890696931, -97.4681822227123, 0.0)",35.257389,-97.468182,0.0
3,3,10 Torr Distilling and Brewing,www.10torr.com,490 Mill St,Reno,NV,"490 Mill St, Reno, NV","(490, Mill Street, Midtown, Reno, Washoe Count...","(39.52570391836734, -119.80577028571429, 0.0)",39.525704,-119.805770,0.0
4,4,10-56 Brewing Company,,2120 US-35,Knox,IN,"2120 US-35, Knox, IN","(South Heaton Street, Knox, Starke County, Ind...","(41.2784872, -86.6206672, 0.0)",41.278487,-86.620667,0.0
6,6,1035 Brewing,,1035 S Central Ave,Sidney,MT,"1035 S Central Ave, Sidney, MT","(1035 Brewing, 1035, South Central Avenue, Fou...","(47.7070485, -104.1643577, 0.0)",47.707048,-104.164358,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8399,8399,ZwanzigZ Brewing,www.zwanzigz.com,1038 Lafayette Ave,Columbus,IN,"1038 Lafayette Ave, Columbus, IN","(ZwanzigZ Pizza & Brewing, 1038, Lafayette Ave...","(39.20926005, -85.91909804812639, 0.0)",39.209260,-85.919098,0.0
8400,8400,Zwei Brewing Co,www.zweibruderbrewing.com,4612 S Mason St Ste 120,Fort Collins,CO,"4612 S Mason St Ste 120, Fort Collins, CO","(proVelo, 4612, South Mason Street, Arbor Plaz...","(40.5227407, -105.07859744351421, 0.0)",40.522741,-105.078597,0.0
8401,8401,Zydeco Brew Werks,,1902 E 7th Ave,Tampa,FL,"1902 E 7th Ave, Tampa, FL","(1902, East 7th Avenue, Ybor City, Tampa, Hill...","(27.960343795918366, -82.4378322857143, 0.0)",27.960344,-82.437832,0.0
8403,8403,Zymurgy Brewing Co,www.zymurgybrew.com,624 Main St East,Menomonie,WI,"624 Main St East, Menomonie, WI","(624, Main Street East, South Menomonie, Menom...","(44.876363, -91.92250454511311, 0.0)",44.876363,-91.922505,0.0


In [121]:
BrewAssoc_df_split = BrewAssoc_df_split.drop(['location','point'],axis=1)
BrewAssoc_df_split

,index,name,website,address,city,state,Address_merg,latitude,longitue,Elevation
0,0,Patton Block Grill and Brewpub,,88-90 Public Square,Monmouth,IL,"88-90 Public Square, Monmouth, IL",40.912282,-90.647964,0.0
1,1,(405) Brewing Co,www.405brewing.com,1716 Topeka St,Norman,OK,"1716 Topeka St, Norman, OK",35.257389,-97.468182,0.0
3,3,10 Torr Distilling and Brewing,www.10torr.com,490 Mill St,Reno,NV,"490 Mill St, Reno, NV",39.525704,-119.805770,0.0
4,4,10-56 Brewing Company,,2120 US-35,Knox,IN,"2120 US-35, Knox, IN",41.278487,-86.620667,0.0
6,6,1035 Brewing,,1035 S Central Ave,Sidney,MT,"1035 S Central Ave, Sidney, MT",47.707048,-104.164358,0.0
...,...,...,...,...,...,...,...,...,...,...
8399,8399,ZwanzigZ Brewing,www.zwanzigz.com,1038 Lafayette Ave,Columbus,IN,"1038 Lafayette Ave, Columbus, IN",39.209260,-85.919098,0.0
8400,8400,Zwei Brewing Co,www.zweibruderbrewing.com,4612 S Mason St Ste 120,Fort Collins,CO,"4612 S Mason St Ste 120, Fort Collins, CO",40.522741,-105.078597,0.0
8401,8401,Zydeco Brew Werks,,1902 E 7th Ave,Tampa,FL,"1902 E 7th Ave, Tampa, FL",27.960344,-82.437832,0.0
8403,8403,Zymurgy Brewing Co,www.zymurgybrew.com,624 Main St East,Menomonie,WI,"624 Main St East, Menomonie, WI",44.876363,-91.922505,0.0


In [1]:
BrewAssoc_df_split

NameError: name 'BrewAssoc_df_split' is not defined

In [111]:
con = sqlite3.connect("RateBeer_BeerAdvocate/beer_advocate_2017/beer_advocate_2017.sqlite")
ratingsAll_df = pd.read_sql_query("SELECT * FROM beer_advocate_2017_ratings", con)
con.close()
ratingsAll_df


,index,brewery_name,beer_name,brewery_beer,beer_style,beer_abv,review_overall
0,0,Vecchio Birraio,Sausa Weizen,Vecchio Birraio Sausa Weizen,Hefeweizen,5.0,1.5
1,1,Vecchio Birraio,Red Moon,Vecchio Birraio Red Moon,English Strong Ale,6.2,3.0
2,2,Vecchio Birraio,Black Horse Black Beer,Vecchio Birraio Black Horse Black Beer,Foreign / Export Stout,6.5,3.0
3,3,Vecchio Birraio,Sausa Pils,Vecchio Birraio Sausa Pils,German Pilsener,5.0,3.0
4,4,Caldera Brewing Company,Cauldron DIPA,Caldera Brewing Company Cauldron DIPA,American Double / Imperial IPA,7.7,4.0
...,...,...,...,...,...,...,...
1586609,1586609,The Defiant Brewing Company,The Horseman's Ale,The Defiant Brewing Company The Horseman's Ale,Pumpkin Ale,5.2,5.0
1586610,1586610,The Defiant Brewing Company,The Horseman's Ale,The Defiant Brewing Company The Horseman's Ale,Pumpkin Ale,5.2,4.0
1586611,1586611,The Defiant Brewing Company,The Horseman's Ale,The Defiant Brewing Company The Horseman's Ale,Pumpkin Ale,5.2,4.5
1586612,1586612,The Defiant Brewing Company,The Horseman's Ale,The Defiant Brewing Company The Horseman's Ale,Pumpkin Ale,5.2,4.0


In [112]:
con = sqlite3.connect("RateBeer_BeerAdvocate/RateBeer_scrape.sqlite")
rateBeer_scrape_df = pd.read_sql_query("SELECT * FROM beer_advocate_2017_ratings", con)
con.close()
rateBeer_scrape_df

,index,Rank,NAME,COUNT,ABV,SCORE,Style
0,0,1.0,Uerige Doppel Sticke,933.0,8.5%,3.76,Altbier / Sticke Alt
1,1,2.0,Schumacher 1838er,117.0,5.0%,3.71,Altbier / Sticke Alt
2,2,3.0,Schlüssel Stike,12.0,6.0%,3.67,Altbier / Sticke Alt
3,3,4.0,Godspeed Oberkassel,18.0,4.4%,3.63,Altbier / Sticke Alt
4,4,5.0,Vrooden Doppelsticke Altbier,16.0,8.5%,3.63,Altbier / Sticke Alt
...,...,...,...,...,...,...,...
6048,45,46.0,Klostergården Alstadberger,99.0,6.5%,3.50,Traditional Ale - Other
6049,46,47.0,WRCLW Schöps Smoked Plums Porto BA,27.0,4.8%,3.50,Traditional Ale - Other
6050,47,48.0,Olvalde Brynhildr's Gift,51.0,8.2%,3.50,Traditional Ale - Other
6051,48,49.0,Blaugies Bière Darbyste,563.0,5.8%,3.49,Traditional Ale - Other


In [123]:
engine = create_engine('sqlite:///mergeDashboard.sqlite', echo=False)
sqlite_connection = engine.connect()

BrewAssoc_df_split.to_sql("Brewers_Association", sqlite_connection)
ratingsAll_df.to_sql("Beer_Advocate",sqlite_connection)
rateBeer_scrape_df.to_sql("Rate_Beer",sqlite_connection)
sqlite_connection.close()


ValueError: Table 'Brewers_Association' already exists.

In [5]:
con = sqlite3.connect("hawaii.sqlite")
BrewAssoc = pd.read_sql_query("SELECT * FROM measurement", con)
BrewAssoc

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0
...,...,...,...,...,...
19545,19546,USC00516128,2017-08-19,0.09,71.0
19546,19547,USC00516128,2017-08-20,NaN,78.0
19547,19548,USC00516128,2017-08-21,0.56,76.0
19548,19549,USC00516128,2017-08-22,0.50,76.0


In [127]:
ratingsAll = pd.read_sql_query("SELECT * FROM Beer_Advocate", con)
ratingsAll.head()

,level_0,index,brewery_name,beer_name,brewery_beer,beer_style,beer_abv,review_overall
0,0,0,Vecchio Birraio,Sausa Weizen,Vecchio Birraio Sausa Weizen,Hefeweizen,5.0,1.5
1,1,1,Vecchio Birraio,Red Moon,Vecchio Birraio Red Moon,English Strong Ale,6.2,3.0
2,2,2,Vecchio Birraio,Black Horse Black Beer,Vecchio Birraio Black Horse Black Beer,Foreign / Export Stout,6.5,3.0
3,3,3,Vecchio Birraio,Sausa Pils,Vecchio Birraio Sausa Pils,German Pilsener,5.0,3.0
4,4,4,Caldera Brewing Company,Cauldron DIPA,Caldera Brewing Company Cauldron DIPA,American Double / Imperial IPA,7.7,4.0


In [128]:
rateBeer_scrape = pd.read_sql_query("SELECT * FROM Rate_Beer", con)
rateBeer_scrape.head()

,level_0,index,Rank,NAME,COUNT,ABV,SCORE,Style
0,0,0,1.0,Uerige Doppel Sticke,933.0,8.5%,3.76,Altbier / Sticke Alt
1,1,1,2.0,Schumacher 1838er,117.0,5.0%,3.71,Altbier / Sticke Alt
2,2,2,3.0,Schlüssel Stike,12.0,6.0%,3.67,Altbier / Sticke Alt
3,3,3,4.0,Godspeed Oberkassel,18.0,4.4%,3.63,Altbier / Sticke Alt
4,4,4,5.0,Vrooden Doppelsticke Altbier,16.0,8.5%,3.63,Altbier / Sticke Alt


In [129]:
result = BrewAssoc_df.to_json(orient="records")


BrewAssoc_df.to_json(r'BrewAssoc.json', orient="records")

In [55]:
sqlite_connection.close()